In [1]:
# pip install tk

In [2]:
import numpy as np
from tkinter import *
from PIL import Image, ImageTk
import cv2
import pandas as pd

#Grid turns to 0 when calling the functions in submit button
#Try and figure out how to access items in the dataframe, i want to get all values in specific column
#Not printing matches

In [3]:
def make_hold_array(x_coordinates, y_coordinates):
    grid = np.meshgrid(x_coordinates,y_coordinates)
    grid = np.array(grid)
    holds = grid.reshape(2,-1).T
    
    return holds

def make_hold_grid():
    hand_x = [55, 119, 183, 246, 310, 375, 440, 503, 568, 632,697]
    hand_y = [113,178,242,304,369,431,495,560,625,689,753,817,880,945,1008,1076,1138,1203,1267]
    foot_right_x = [88,217,344,473,597,729]
    foot_left_x = [25,157,281,411,539,665]
    foot_right_y = [275,405,530,656,784,917,1047,1171]
    foot_left_y = [338,464,595,723,847,976,1106]
    foot_bottom_x = [25,88,149,215,276,341,405,470,534,599,663,727]
    foot_bottom_y = [1300]
    
    hands = make_hold_array(hand_x,hand_y)
    foot_left = make_hold_array(foot_left_x,foot_left_y)
    foot_right = make_hold_array(foot_right_x,foot_right_y)
    foot_bottom = make_hold_array(foot_bottom_x,foot_bottom_y)
    
    holds = np.append(hands,foot_left,axis=0)
    holds = np.append(holds,foot_right,axis=0)
    holds = np.append(holds,foot_bottom,axis=0)
    
    #sort holds, y value first then x, so we sort by row, then column
    holds = holds[np.lexsort((holds[:,0],holds[:,1]))]

    return holds

In [4]:
holds = make_hold_grid()


def find_id(input_cord, holds = holds):
    """Given a coordinate (x,y) This function finds the closest hold out of a list of known holds, and returns the hold
    number"""
    
    x_dist = (input_cord[0] - holds[:, 0]) #All rows, 0th element in each row
    y_dist = (input_cord[1] - holds[:, 1]) #All rows, first element in each row
    distance = (x_dist**2) + (y_dist**2) #diagonal distance
    return np.argmin(distance)

In [5]:
print(repr(holds))

array([[  55,  113],
       [ 119,  113],
       [ 183,  113],
       [ 246,  113],
       [ 310,  113],
       [ 375,  113],
       [ 440,  113],
       [ 503,  113],
       [ 568,  113],
       [ 632,  113],
       [ 697,  113],
       [  55,  178],
       [ 119,  178],
       [ 183,  178],
       [ 246,  178],
       [ 310,  178],
       [ 375,  178],
       [ 440,  178],
       [ 503,  178],
       [ 568,  178],
       [ 632,  178],
       [ 697,  178],
       [  55,  242],
       [ 119,  242],
       [ 183,  242],
       [ 246,  242],
       [ 310,  242],
       [ 375,  242],
       [ 440,  242],
       [ 503,  242],
       [ 568,  242],
       [ 632,  242],
       [ 697,  242],
       [  88,  275],
       [ 217,  275],
       [ 344,  275],
       [ 473,  275],
       [ 597,  275],
       [ 729,  275],
       [  55,  304],
       [ 119,  304],
       [ 183,  304],
       [ 246,  304],
       [ 310,  304],
       [ 375,  304],
       [ 440,  304],
       [ 503,  304],
       [ 568,

In [6]:
def display_template(template_type,hold,tl_x,tl_y):
    global my_canvas
    if template_type == 'diamond':
        tl_x -=38
        tl_y -=37
    else:
        tl_x -=33
        tl_y -=33
    
    photo = ImageTk.PhotoImage(file=f'{template_type}.png')
    img = my_canvas.create_image(tl_x,tl_y,image=photo,anchor='nw',tag='templates')
    img_grid[hold] = img
    label = Label(image=photo)
    label.image = photo # keep a reference!
    label.pack()

In [7]:
def find_names(grid):
    input_grid = grid.copy()
    for i in range(len(input_grid)):
            if input_grid[i] == '-':
                input_grid[i] = 0
            elif input_grid[i] == 'Hand':
                input_grid[i] = 2
            elif input_grid[i] == 'Foot':
                input_grid[i] = 3
            elif input_grid[i] == 'Start':
                input_grid[i] = 1
            elif input_grid[i] == 'Finish':
                input_grid[i] = 4

    train_data = pd.read_csv("kilter_lookup.csv")

    data = {}
    [data.update({f"Hold {i}":input_grid[i]}) for i in range(len(input_grid))]
    my_data = pd.DataFrame([data])
    my_data = my_data.loc[:, (my_data != 0).any(axis=0)]

    original_data = pd.DataFrame()

    for x in my_data.columns:

    #     print(f"my data {int(my_data[x])}")
    #     print(f"found data {train_data[x]}")
        answer_data = train_data.loc[train_data[x] == int(my_data[x])]
        original_data = original_data.append(answer_data)

    output = [x for x in pd.merge(train_data, original_data, on=original_data.columns.tolist(), how='inner')['name']]

    result = sorted(output, key = output.count,
                                    reverse = True)
    
    result = [x for x in result if str(x) != 'nan'] #remove nan
    result = [x for x in result if str(x) != ''] #remove blank
    result = [x for x in result if str(x) != ' '] #remove space
    
    data = {}
    for item in result:
        data.update({item:f"{result.count(item)}"})

#     count = result.count(result[0])
    
#     result = [i for n, i in enumerate(result) if i not in result[:n]] #remove duplicates
    
    
    return data
    
#     data = {}
#     [data.update({f"Hold {i}":input_grid[i]}) for i in range(len(input_grid))]
#     my_data = pd.DataFrame([data])
#     return [x for x in pd.merge(train_data, my_data, on=my_data.columns.tolist(), how='inner')['name']]

In [8]:
grid = ['-' for x in range(len(holds))]
img_grid = ['' for x in range(len(holds))]
hand_location = []
foot_location = []
start_location = []
finish_location = []



SQUARE_HEIGHT = 30

root = Tk()
root.title('Codemy.com - Set Image as Background')
root.iconbitmap('kilter_app.ico')
root.geometry("757x1355")

# scrollbar = Scrollbar(root)
# scrollbar.pack( side = RIGHT, fill = Y )

# Define image
bg = PhotoImage(file='routeset_menu.png')

# Create a canvas
my_canvas = Canvas(root, width=757, height=1355)
my_canvas.pack(fill="both", expand=True)

# Set image in canvas
my_canvas.create_image(0,0, image=bg, anchor="nw")

# scrollbar.config( command = my_canvas.yview )


images = []  # to hold the newly created image"

def getorigin(eventorigin):
#     global x,y
    x = eventorigin.x
    y = eventorigin.y
    
    if y < 69: #If clicking too high (where blue menu bar)
        return
    
#     print(x,y)
    
    hold = find_id((x,y))
#     print(hold)
    
    close_x = holds[find_id((x,y))][0]
    close_y = holds[find_id((x,y))][1]
    
#     circle = ImageTk.PhotoImage(file='circle.png')
#     img = my_canvas.create_image(tl_x,tl_y,image=circle,anchor='nw',tag='templates')
    
#     img_grid[hold] = img
    
#     label = Label(image=circle)
#     label.image = circle # keep a reference!
#     label.pack()
    
    
#     grid[hold] +=1
    
    hand_length = len(hand_location)
    foot_length = len(foot_location)
    start_length = len(start_location)
    finish_length = len(finish_location)
    total_length = hand_length + foot_length + start_length + finish_length

    clicked_button = grid[hold]
#     print(clicked_button)

    if clicked_button == '-':
        if total_length >=35:
            print("Hold Limit reached")
            return
        else:
            #Turn to hand
            grid[hold] = 'Hand'
            display_template('circle',hold,close_x,close_y)
            hand_location.append(hold)

    elif clicked_button == 'Hand':
        #Turn to foot
        hand_location.remove(hold)
        grid[hold] = 'Foot'
        my_canvas.delete(img_grid[hold])
        display_template('diamond',hold,close_x,close_y)
        foot_location.append(hold)

    elif clicked_button == 'Foot':
        foot_location.remove(hold)
        my_canvas.delete(img_grid[hold])
        if start_length >= 2:
            if finish_length >=2:
                #Skip Start & Finish, turn to blank
                grid[hold] = '-'
            else: 
                #Skip start, turn to finish
                grid[hold] = 'Finish'
                display_template('triangle',hold,close_x,close_y)
                finish_location.append(hold)
        else:
            #Turn to start
            grid[hold] = 'Start'
            display_template('square',hold,close_x,close_y)
            start_location.append(hold)

    elif clicked_button == 'Start':
        start_location.remove(hold)
        my_canvas.delete(img_grid[hold])
        if finish_length >=2:
                #Skip Finish, turn to blank
                grid[hold] = '-'
        else: 
            #Turn to finish
            grid[hold] = 'Finish'
            display_template('triangle',hold,close_x,close_y)
            finish_location.append(hold)

    elif clicked_button == 'Finish':
        #Turn to blank
        finish_location.remove(hold)
        grid[hold] = '-'
        my_canvas.delete(img_grid[hold])
    

root.bind("<Button 1>",getorigin)


def submit():
    print(grid)
#     print('Submit')
    data = find_names(grid)
    matches = [*data]
    
#     print(f"array {matches}")
    print(f"length {len(matches)}")
#     [print(x) for x in matches]
#     print(*matches, sep=", ")
#     print(matches[63])

    top= Toplevel(root)
    
    
    top.geometry("250x250")
    top.title("Search Results")
    
    first_result = matches[0]
    
    Label(top, text= f"The Best Match ({data[first_result]} hold matches): {first_result} ").place(x=10,y=10)

    Lb1 = Listbox(top)
    
    for x in range(1,len(matches)):
        Lb1.insert(x,f"({data[matches[x]]}) {matches[x]}")
    Lb1.pack(expand=True)
    

    
#     if len(matches) <= 0:
#         Label(top, text= f"Found Nothing").place(x=10,y=10)
#     elif len(matches) == 1:
#         Label(top, text= f"Found {len(matches)} Match : {match[0]}").place(x=10,y=10)
#     else:
    
    
def reset():
    global grid, img_grid,hand_location,foot_location,start_location,finish_location
    my_canvas.delete('templates')
    grid = ['-' for x in range(len(holds))]
    img_grid = ['' for x in range(len(holds))]
    hand_location = []
    foot_location = []
    start_location = []
    finish_location = []
    
def exit():
#     print('Exit')
    root.destroy()

# add some buttons
button1 = Button(root, text="Submit",command=submit)
button1.config(width=14, height=3)
button2 = Button(root, text="Reset",command=reset)
button2.config(width=10, height=3)
button3 = Button(root, text="Exit",command=exit)
button3.config(width=7, height=3)

button1_window = my_canvas.create_window(750, 8, anchor="ne", window=button1)
button2_window = my_canvas.create_window(420, 8, anchor="ne", window=button2)
button3_window = my_canvas.create_window(8, 8, anchor="nw", window=button3)



root.mainloop()

['-', '-', '-', '-', 'Hand', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Hand', '-', '-', '-', '-', '-', 'Hand', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Hand', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Hand', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Hand', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '

In [9]:
# grid = ['-', '-', 'Finish', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Hand', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Hand', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
# grid = [0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [10]:
grid = ['-', '-', 'Start', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Hand', '-', '-', '-', '-', '-', 'Hand', '-', 'Finish', '-', 'Foot', '-', '-', '-', '-', '-', '-', '-', '-', 'Foot', '-', '-', '-', '-', 'Hand', 'Foot', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Hand', '-', 'Foot', '-', '-', '-', 'Finish', '-', '-', '-', '-', '-', '-', '-', '-', 'Start', '-', '-', '-', '-', 'Foot', '-', 'Hand', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Hand', '-', 'Hand', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Foot', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Hand', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Foot', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']

In [11]:
def make_string(grid):
    
    input_grid = grid.copy()
    
    for i in range(len(input_grid)):
            if input_grid[i] == '-':
                input_grid[i] = 0
            elif input_grid[i] == 'Hand':
                input_grid[i] = 2
            elif input_grid[i] == 'Foot':
                input_grid[i] = 3
            elif input_grid[i] == 'Start':
                input_grid[i] = 1
            elif input_grid[i] == 'Finish':
                input_grid[i] = 4
                
    string = ''.join(str(x) for x in input_grid)
    
    return string

In [12]:
def make_list(string):
    return [x for x in string]

In [13]:
print(grid)
string = make_string(grid)
print(string)
my_list = make_list(string)
print(my_list)

['-', '-', 'Start', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Hand', '-', '-', '-', '-', '-', 'Hand', '-', 'Finish', '-', 'Foot', '-', '-', '-', '-', '-', '-', '-', '-', 'Foot', '-', '-', '-', '-', 'Hand', 'Foot', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Hand', '-', 'Foot', '-', '-', '-', 'Finish', '-', '-', '-', '-', '-', '-', '-', '-', 'Start', '-', '-', '-', '-', 'Foot', '-', 'Hand', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Hand', '-', 'Hand', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Foot', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Hand', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Foot', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '